In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path
from scripts.cross_match_scripts import my_scaler_forward
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns', 500)
import seaborn as sns
import numpy as np
%matplotlib inline

matplotlib settings set


matplotlib settings set


# Catalogs loading and preprocessing

In [3]:
photo_cat = pd.read_pickle(data_path+'csc_allsky_desi_r30_gaia_dered_training.gz_pkl', compression='gzip')


photo_cat_ctps = photo_cat[photo_cat['is_counterpart'] == 1]
photo_cat_field = photo_cat[photo_cat['is_counterpart'] == 0]

fraction_to_retain = 0.5
photo_cat_field = photo_cat_field.sample(frac=fraction_to_retain)

photo_cat = pd.concat([photo_cat_ctps, photo_cat_field])

x_ray_flux_lim = 3e-15
photo_cat = photo_cat.query('(csc_flux_05_2 > @x_ray_flux_lim) | is_counterpart == False')
photo_cat = photo_cat.query('csc_dec>32') #ignore DECALS data because LH area is within MzLS+BASS footprint, see https://www.legacysurvey.org/status/
photo_cat = photo_cat.replace([np.inf, -np.inf], np.nan)
photo_cat.sample(5)

,csc_name,csc_ra,csc_dec,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_r_98,csc_flux_05_2,csc_flux_05_2_err,csc_secure,all_mag_g,rel_mag_g,rel_dered_mag_g,rel_flux_corr_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_flux_corr_r,rel_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,csc_r_false,second_closest_dist,closest_dist,is_counterpart
2755859,2CXO J142132.4+525912,215.385319,52.986831,9011,1851,595236,215.388554,52.984590,1,0.538375,0.627324,0.982143,0.166377,-1.169501,37.865400,-185.533520,223.09850,100.811090,43.874004,6.387792,1.649190,0.002140,0.000025,23.139967,22.984499,22.507385,24.445420,NaN,18.554152,NaN,8.041422,6.298625,6.505462,0.420502,-1.501882,1.751477,-0.935290,0.000000,0.00000,0.000000,0.00000,0.000000,0.0000,10.688714,9011_595236_1851,2.184610,1.953394,104.523285,5.764706,91.931117,TRUE,False,False,False,False,False,False,False,2.921174e-15,1.217156e-15,4.625193e-15,3.876498e-15,2.981921e-15,4.771074e-15,3.348899,6.797672e-15,2.721744e-15,True,23.172288,23.172288,23.139967,3.196124e-15,23.006270,23.006270,22.984499,22.519563,22.519563,22.507385,24.447267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.554394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.237587e-15,1.649585e-15,0.155468,0.632582,0.477114,NaN,NaN,NaN,NaN,NaN,NaN,1.319290,5.799629,0.993166,False
1675527,2CXO J095622.3+690446,149.093324,69.079609,9011,3745,639883,149.080880,69.085309,1,2.046805,1.724363,0.845591,-13.864892,0.509931,174.739210,-33638.324000,247.75755,96.835310,36.575375,4.055963,1.120837,0.001613,0.000020,21.465350,21.735336,22.585280,NaN,23.222187,16.892097,NaN,32.217358,16.968582,5.113930,-27.923090,0.539862,7.017467,-150.307790,0.000000,0.00000,0.000000,0.00000,0.000000,0.0000,26.017672,9011_639883_3745,2.259506,2.080000,116.100538,3.028571,21.322944,TRUE,False,False,False,False,False,False,True,2.506316e-15,1.862802e-15,3.149829e-15,3.404638e-16,1.540193e-16,5.269082e-16,3.509504,2.846780e-15,9.474926e-16,True,21.722309,21.722309,21.465350,1.494412e-14,21.908428,21.908428,21.735336,22.682099,22.682099,22.585280,NaN,NaN,NaN,NaN,23.231221,NaN,NaN,NaN,16.894024,16.894024,16.892097,11.720024,NaN,NaN,NaN,NaN,7.070418e-15,1.535383e-15,-0.269986,-1.119930,-0.849944,NaN,NaN,NaN,5.693183,NaN,NaN,0.621919,2.825272,2.380683,False
1348270,2CXO J072021.4+710032,110.089415,71.008961,9011,1841,643804,110.107057,71.013312,1,159.938260,346.888950,498.842500,195.760280,107.172530,144.755920,1062.052200,20.49130,13.586337,13.123672,1.450045,0.683710,0.000483,0.000006,16.895494,16.085783,15.719438,16.765272,17.421465,17.097700,14.934367,723.997600,1278.620500,1807.137200,235.730260,88.617516,3.181880,2.590089,0.976226,422.68378,0.153401,241.71606,-4.132243,161.4222,25.930267,9011_643804_1841,3.174051,2.144583,7.853444,4.222222,23.668151,TRUE,False,False,False,False,False,False,False,0.000000e+00,0.000000e+00,3.055521e-16,1.390373e-15,1.033385e-15,1.747360e-15,4.377449,1.390373e-15,5.901206e-16,True,16.990119,16.990119,16.895494,1.005571e-12,16.149524,16.149524,

In [4]:
photo_cat.is_counterpart.value_counts()

False    244008
True      12452
Name: is_counterpart, dtype: int64

In [5]:
print('fraction of missed measurements per is_counterpart per filter, in per cent:')
photo_cat.groupby('is_counterpart')[['rel_dered_mag_g',	'rel_dered_mag_r',	'rel_dered_mag_z',	'rel_dered_mag_w1',	'rel_dered_mag_w2',	'rel_dered_mag_w3',	'rel_dered_mag_w4']].agg(lambda x: np.round(100*np.mean(np.
isnan(x)),1))

fraction of missed measurements per is_counterpart per filter, in per cent:


,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_mag_w3,rel_dered_mag_w4
is_counterpart,,,,,,,
False,26.6,20.5,12.7,45.0,66.3,95.3,90.7
True,8.8,5.9,1.3,3.1,6.1,67.6,81.3


In [6]:
features_cols = ['rel_dered_mag_g','rel_dered_mag_r','rel_dered_mag_z'] 
features_cols += ['rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z']


features_cols += ['rel_dered_mag_w1', 'rel_dered_mag_w2']
features_cols += ['rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2']

features_cols += ['rel_dered_mag_w3', 'rel_dered_mag_w4']
features_cols += ['rel_dered_z_w3', 'rel_dered_r_w4', 'rel_dered_w3_w4']


target_col = ['is_counterpart']

photo_cat = photo_cat[features_cols+target_col+['csc_flux_05_2']]

#photo_cat= photo_cat.dropna(subset = features_cols, how = 'any')

print(photo_cat.is_counterpart.value_counts())
photo_cat = photo_cat.sample(frac=1)
photo_cat.reset_index(inplace=True, drop = True)

photo_cat_scaled = my_scaler_forward(photo_cat)
photo_cat_scaled.to_pickle('./model/train_test_validation_data.pkl')
photo_cat_scaled.head(15)


False    244008
True      12452
Name: is_counterpart, dtype: int64
rel_dered_mag_g scaled by 1/35
rel_dered_mag_r scaled by 1/35
rel_dered_mag_z scaled by 1/35
rel_dered_g_r scaled by 1/10
rel_dered_r_z scaled by 1/10
rel_dered_g_z scaled by 1/10
rel_dered_mag_w1 scaled by 1/35
rel_dered_mag_w2 scaled by 1/35
rel_dered_z_w1 scaled by 1/10
rel_dered_r_w2 scaled by 1/10
rel_dered_w1_w2 scaled by 1/10
rel_dered_mag_w3 scaled by 1/35
rel_dered_mag_w4 scaled by 1/35
rel_dered_z_w3 scaled by 1/10
rel_dered_r_w4 scaled by 1/10
rel_dered_w3_w4 scaled by 1/10


,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_g_r,rel_dered_r_z,rel_dered_g_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_mag_w3,rel_dered_mag_w4,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,is_counterpart,csc_flux_05_2
0,NaN,NaN,0.663283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1.107205e-15
1,NaN,NaN,0.633139,NaN,NaN,NaN,0.549803,0.547063,0.291675,NaN,0.009592,NaN,NaN,NaN,NaN,NaN,False,1.401215e-15
2,0.664455,0.650994,0.626818,0.047114,0.084617,0.131731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.000000e+00
3,0.668100,0.641932,0.624469,0.091587,0.061123,0.152709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,6.234301e-15
4,0.675134,0.651074,0.651487,0.084210,-0.001443,0.082767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,5.152043e-16
5,0.612615,0.615862,0.618101,-0.011365,-0.007837,-0.019202,0.609356,0.594757,0.030609,0.073868,0.051096,NaN,NaN,NaN,NaN,NaN,False,5.945183e-15
6,NaN,0.669941,0.642364,NaN,0.096520,NaN,0.598414,NaN,0.153823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,9.287604e-16
7,0.648836,0.620341,0.609426,0.099732,0.038203,0.137935,0.610231,NaN,-0.002819,NaN,NaN,0.501348,0.433429,0.378273,0.654190,0.237714,False,7.486776e-16
8,NaN,0.672279,0.649562,NaN,0.079508,NaN,0.598312,NaN,0.179377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1.123437e-14
9,NaN,NaN,0.648937,NaN,NaN,NaN,0.604140,0.605410,0.156789,NaN,-0.004444,NaN,NaN,NaN,NaN,NaN,False,0.000000e+00
